# 평균속도 및 교통량
- 2017~2018
- 대구 빅데이터 활용센터
- 4-2. 도로정보 및 교통량

### 법정동 행정동 mapper
- ~2021
- 대구 빅데이터 활용센터
- 9-1. 유동인구 데이터

In [675]:
import pandas as pd
import numpy as np
from IPython.display import display
from copy import deepcopy

In [676]:
car_speed = pd.read_csv('./externals/대구_빅데이터_활용센터/4_2_car_speed.csv', encoding='cp949', index_col=0).reset_index(drop=True)
car_traffic = pd.read_csv('./externals/대구_빅데이터_활용센터/4_2_car_traffic.csv', encoding='cp949', index_col=0).reset_index(drop=True)

In [677]:
car_speed.head()

,도명,시군구명,읍면동명,road_rank,도로명,시간,연장,연장.1,전체평균속도,전체평균속도.1
0,NaN,NaN,NaN,NaN,NaN,NaN,count,sum,count,sum
1,대구광역시,남구,대명10동,일반국도,성당로,0.0,2,0.612,2,89.70608585400001
2,대구광역시,남구,대명10동,일반국도,성당로,1.0,2,0.612,2,94.941135349
3,대구광역시,남구,대명10동,일반국도,성당로,2.0,2,0.612,2,95.273503559
4,대구광역시,남구,대명10동,일반국도,성당로,3.0,2,0.612,2,100.538239033


In [678]:
car_traffic.head()

,시도명,시군구명,읍면동명,road_rank,도로명,시간,연장,연장.1,전체,전체.1,승용차,승용차.1,버스,버스.1,트럭,트럭.1
0,NaN,NaN,NaN,NaN,NaN,NaN,count,sum,count,sum,count,sum,count,sum,count,sum
1,대구광역시,남구,대명10동,일반국도,성당로,0.0,2,0.612,2,942.0,2,764.0,2,56.0,2,122.0
2,대구광역시,남구,대명10동,일반국도,성당로,1.0,2,0.612,2,715.0,2,565.0,2,39.0,2,111.0
3,대구광역시,남구,대명10동,일반국도,성당로,2.0,2,0.612,2,590.0,2,457.0,2,30.0,2,103.0
4,대구광역시,남구,대명10동,일반국도,성당로,3.0,2,0.612,2,521.0,2,397.0,2,26.0,2,98.0


In [679]:
df_cars = {
  'speed': car_speed,
  'traffic': car_traffic
}

# 컬럼명 변경
- sum / count

In [680]:
for k,v in df_cars.items():
  new_col = [col if pd.isna(v.loc[0, col]) else f'{col.split(".")[0]}_{v.loc[0, col]}' for col in v.columns]
  new_col = [col if col != '시도명' else '도명' for col in v.columns]
  v.columns = new_col
  df_cars[k] = v.drop(0).drop(columns=['도명']).reset_index(drop=True)

In [681]:
for k,v in df_cars.items():
  print(k)
  display(v.head(2))

speed


,시군구명,읍면동명,road_rank,도로명,시간,연장,연장.1,전체평균속도,전체평균속도.1
0,남구,대명10동,일반국도,성당로,0.0,2,0.612,2,89.70608585400001
1,남구,대명10동,일반국도,성당로,1.0,2,0.612,2,94.941135349


traffic


,시군구명,읍면동명,road_rank,도로명,시간,연장,연장.1,전체,전체.1,승용차,승용차.1,버스,버스.1,트럭,트럭.1
0,남구,대명10동,일반국도,성당로,0.0,2,0.612,2,942.0,2,764.0,2,56.0,2,122.0
1,남구,대명10동,일반국도,성당로,1.0,2,0.612,2,715.0,2,565.0,2,39.0,2,111.0


# 읍면동명 안 맞는 것 수정
- · -> .
- 안심3.4동 -> 안심3동 안심4동
- 옥포면 -> 옥포읍
- 현풍면 -> 현풍읍
- 유가면 -> 유가읍

In [682]:
for k,v in df_cars.items():
  v['읍면동명'] = v['읍면동명'].str.replace('·', '.')
  df_cars[k] = v

In [683]:
for k,v in df_cars.items():
  v['읍면동명'] = v['읍면동명'].str.replace('옥포면', '옥포읍').str.replace('현풍면', '현풍읍').str.replace('유가면', '유가읍')
  df_cars[k] = v

In [684]:
for k,v in df_cars.items():
  ansim_4_dong = []
  rows = v.loc[v['읍면동명'] == '안심3.4동']
  for idx, row in rows.iterrows():
    ansim_4_dong.append({**row, '읍면동명': '안심3동'})
    ansim_4_dong.append({**row, '읍면동명': '안심4동'})
  v = v.drop(v.loc[v['읍면동명'] == '안심3.4동'].index)
  v = pd.concat([v, pd.DataFrame(ansim_4_dong)]).reset_index(drop=True)
  df_cars[k] = v

## 법정동 행정동 mapper

In [685]:
dong_mapper = pd.read_csv('./externals/대구_빅데이터_활용센터/9_1_dong_mapper.csv', index_col=0).drop(columns=['시도명'])
dong_mapper

,읍면동명,동리명,생성일자
0,동인동,동인동1가,2011
1,동인동,동인동2가,2011
2,동인동,동인동3가,2011
3,동인동,동인동4가,2011
4,삼덕동,삼덕동1가,1998
...,...,...,...
390,구지면,화산리,1995
391,구지면,수리리,1995
392,구지면,징리,1995
393,구지면,오설리,1995


In [686]:
def get_list_of_ldongs(dong_name: str) -> np.ndarray:
  return dong_mapper.loc[dong_mapper['읍면동명'] == dong_name]['동리명'].values

get_list_of_ldongs('유가읍')

array(['유가읍', '음리', '양리', '용리', '봉리', '쌍계리', '초곡리', '상리', '금리', '유곡리',
       '도의리', '가태리', '한정리', '본말리'], dtype=object)

In [687]:
def map_dongs(row: pd.Series):
  row = {**row}
  list_of_ldongs = get_list_of_ldongs(row['읍면동명'])
  if len(list_of_ldongs) == 0:
    print(row)
  return [{**row, '동': dong} for dong in list_of_ldongs]

map_dongs(df_cars['speed'].loc[620])

[{'시군구명': '남구',
  '읍면동명': '봉덕2동',
  'road_rank': '도시고속도로',
  '도로명': '신천대로',
  '시간': 20.0,
  '연장': '5',
  '연장.1': '1.495',
  '전체평균속도': '5',
  '전체평균속도.1': '278.901504292',
  '동': '봉덕동'}]

In [688]:
for k,v in df_cars.items():
  new_df = []
  for idx, row in v.iterrows():
    new_df.extend(map_dongs(row))
  df_cars[k] = pd.DataFrame(new_df)

In [689]:
df_cars['speed'].head(2)

,시군구명,읍면동명,road_rank,도로명,시간,연장,연장.1,전체평균속도,전체평균속도.1,동
0,남구,대명10동,일반국도,성당로,0.0,2,0.612,2,89.70608585400001,대명동
1,남구,대명10동,일반국도,성당로,1.0,2,0.612,2,94.941135349,대명동


In [690]:
df_cars['traffic'].head(2)

,시군구명,읍면동명,road_rank,도로명,시간,연장,연장.1,전체,전체.1,승용차,승용차.1,버스,버스.1,트럭,트럭.1,동
0,남구,대명10동,일반국도,성당로,0.0,2,0.612,2,942.0,2,764.0,2,56.0,2,122.0,대명동
1,남구,대명10동,일반국도,성당로,1.0,2,0.612,2,715.0,2,565.0,2,39.0,2,111.0,대명동


In [691]:
df_cars_origin = deepcopy(df_cars)

In [692]:
for k,v in df_cars.items():
  dong = v['동']
  time = v['시간'].map(int)
  gu = v['시군구명']
  v = v.drop(columns=['읍면동명', 'road_rank', '도로명', '동', '시간', '시군구명'])
  v = v.map(float)
  v['시간'] = time
  v['동'] = dong
  v['시군구명'] = gu
  # v = v.groupby(['시간', '동']).sum()
  df_cars[k] = v
  display(v.head())
  

,연장,연장.1,전체평균속도,전체평균속도.1,시간,동,시군구명
0,2.0,0.612,2.0,89.706086,0,대명동,남구
1,2.0,0.612,2.0,94.941135,1,대명동,남구
2,2.0,0.612,2.0,95.273504,2,대명동,남구
3,2.0,0.612,2.0,100.538239,3,대명동,남구
4,2.0,0.612,2.0,96.398821,4,대명동,남구


,연장,연장.1,전체,전체.1,승용차,승용차.1,버스,버스.1,트럭,트럭.1,시간,동,시군구명
0,2.0,0.612,2.0,942.0,2.0,764.0,2.0,56.0,2.0,122.0,0,대명동,남구
1,2.0,0.612,2.0,715.0,2.0,565.0,2.0,39.0,2.0,111.0,1,대명동,남구
2,2.0,0.612,2.0,590.0,2.0,457.0,2.0,30.0,2.0,103.0,2,대명동,남구
3,2.0,0.612,2.0,521.0,2.0,397.0,2.0,26.0,2.0,98.0,3,대명동,남구
4,2.0,0.612,2.0,547.0,2.0,419.0,2.0,28.0,2.0,100.0,4,대명동,남구


# 평균값 구하기
- col.1 / col

In [693]:
for k, v in df_cars.items():
  sum_cols = v.iloc[:, 1:-3:2]
  cnt_cols = v.iloc[:, 0:-3:2]
  if k == 'speed':
    sum_cols.columns = ['연장', '평균속도']
    cnt_cols.columns = ['연장', '평균속도']
  else:
    sum_cols.columns = ['연장',	'전체',	'승용차',	'버스',	'트럭']
    cnt_cols.columns = ['연장',	'전체',	'승용차',	'버스',	'트럭']
  
  mean_cols = sum_cols / cnt_cols  
  df_cars[k] = pd.concat([v.iloc[:,-3:], mean_cols], axis=1)
  display(df_cars[k].head())
  

,시간,동,시군구명,연장,평균속도
0,0,대명동,남구,0.306,44.853043
1,1,대명동,남구,0.306,47.470568
2,2,대명동,남구,0.306,47.636752
3,3,대명동,남구,0.306,50.269120
4,4,대명동,남구,0.306,48.199410


,시간,동,시군구명,연장,전체,승용차,버스,트럭
0,0,대명동,남구,0.306,471.0,382.0,28.0,61.0
1,1,대명동,남구,0.306,357.5,282.5,19.5,55.5
2,2,대명동,남구,0.306,295.0,228.5,15.0,51.5
3,3,대명동,남구,0.306,260.5,198.5,13.0,49.0
4,4,대명동,남구,0.306,273.5,209.5,14.0,50.0


# 시간별 동별 평균속도 구하기
- 연장은 당장은 필요 없을 것 같으므로 뺍니다.

In [694]:
for k, v in df_cars.items():
  v = v.groupby(['시군구명', '동', '시간']).mean()
  v = v.drop(columns=['연장'])
  df_cars[k] = v.reset_index()
  display(v)

평균속도
시군구명 동   시간           
남구   대명동 0   38.864627
         1   40.851875
         2   41.903996
         3   42.790881
         4   41.755621
...                ...
중구   화전동 19  25.754427
         20  29.728188
         21  30.701109
         22  32.399050
         23  35.848125

[6984 rows x 1 columns]

전체         승용차         버스         트럭
시군구명 동   시간                                               
남구   대명동 0    452.243651  384.119955  26.441345  41.682351
         1    356.607181  302.249509  19.941232  34.416440
         2    306.036168  259.095238  16.531293  30.409637
         3    275.725510  233.082880  14.638549  28.004082
         4    275.713870  233.017989  14.680612  28.015268
...                  ...         ...        ...        ...
중구   화전동 19  1084.583333  927.972222  75.111111  81.500000
         20   932.888889  798.444444  61.472222  72.972222
         21   889.000000  761.027778  57.472222  70.500000
         22   756.916667  647.555556  46.222222  63.138889
         23   590.444444  504.138889  33.361111  52.944444

[6984 rows x 4 columns]

# 저장값
- car_traffic.csv
- car_speed.csv

In [695]:
for k, v in df_cars.items():
  v.reset_index().drop(columns=['index']).to_csv(f'../../main/data/origin/additionals/car_{k}.csv')

# 추가 자료
- 동별 도로등급별 개수 및 연장(길이) 

In [696]:
for k,v in df_cars_origin.items():
  display(v.head(3))

,시군구명,읍면동명,road_rank,도로명,시간,연장,연장.1,전체평균속도,전체평균속도.1,동
0,남구,대명10동,일반국도,성당로,0.0,2,0.612,2,89.70608585400001,대명동
1,남구,대명10동,일반국도,성당로,1.0,2,0.612,2,94.941135349,대명동
2,남구,대명10동,일반국도,성당로,2.0,2,0.612,2,95.273503559,대명동


,시군구명,읍면동명,road_rank,도로명,시간,연장,연장.1,전체,전체.1,승용차,승용차.1,버스,버스.1,트럭,트럭.1,동
0,남구,대명10동,일반국도,성당로,0.0,2,0.612,2,942.0,2,764.0,2,56.0,2,122.0,대명동
1,남구,대명10동,일반국도,성당로,1.0,2,0.612,2,715.0,2,565.0,2,39.0,2,111.0,대명동
2,남구,대명10동,일반국도,성당로,2.0,2,0.612,2,590.0,2,457.0,2,30.0,2,103.0,대명동


In [697]:
for k,v in df_cars_origin.items():
  v = v.iloc[:, [0,1,2,4,5,6,-1]]
  road_rank = v['road_rank']
  dong = v['동']
  gu = v['시군구명']
  v = v.iloc[:,[4,5]].map(float)
  v['동'] = dong
  v['시군구명'] = gu
  v['도로등급'] = road_rank
  v = v.rename(columns={'연장': '연장_count', '연장.1': '연장_mean'})
  v = v.groupby(['시군구명', '동', '도로등급']).mean().drop(columns=['연장_count']).reset_index()
  df_cars[k] = v
  display(v.head(3))

,시군구명,동,도로등급,연장_mean
0,남구,대명동,일반국도,0.902333
1,남구,대명동,특별광역시도,1.858611
2,남구,봉덕동,도시고속도로,0.887500


,시군구명,동,도로등급,연장_mean
0,남구,대명동,일반국도,0.902333
1,남구,대명동,특별광역시도,1.858611
2,남구,봉덕동,도시고속도로,0.887500


In [698]:
df_road = df_cars['traffic']
df_road

,시군구명,동,도로등급,연장_mean
0,남구,대명동,일반국도,0.902333
1,남구,대명동,특별광역시도,1.858611
2,남구,봉덕동,도시고속도로,0.887500
3,남구,봉덕동,연결로,0.783500
4,남구,봉덕동,특별광역시도,1.296455
...,...,...,...,...
716,중구,태평로3가,특별광역시도,1.097667
717,중구,포정동,특별광역시도,1.112667
718,중구,하서동,특별광역시도,1.186000
719,중구,향촌동,특별광역시도,1.186000


In [699]:
df_road_total_long = df_road.drop(columns=['도로등급']).groupby(['시군구명', '동']).sum().reset_index()
df_road_total_long.columns = ['구', '동', '동별_총연장']
df_road_total_long

,구,동,동별_총연장
0,남구,대명동,2.760944
1,남구,봉덕동,2.967455
2,남구,이천동,6.376500
3,달서구,갈산동,4.851884
4,달서구,감삼동,3.303000
...,...,...,...
286,중구,태평로3가,1.097667
287,중구,포정동,1.112667
288,중구,하서동,1.186000
289,중구,향촌동,1.186000


In [700]:
df_road = df_road.merge(df_road_total_long, how='left', on='동')
df_road

,시군구명,동,도로등급,연장_mean,구,동별_총연장
0,남구,대명동,일반국도,0.902333,남구,2.760944
1,남구,대명동,특별광역시도,1.858611,남구,2.760944
2,남구,봉덕동,도시고속도로,0.887500,남구,2.967455
3,남구,봉덕동,연결로,0.783500,남구,2.967455
4,남구,봉덕동,특별광역시도,1.296455,남구,2.967455
...,...,...,...,...,...,...
729,중구,태평로3가,특별광역시도,1.097667,중구,1.097667
730,중구,포정동,특별광역시도,1.112667,중구,1.112667
731,중구,하서동,특별광역시도,1.186000,중구,1.186000
732,중구,향촌동,특별광역시도,1.186000,중구,1.186000


In [701]:
df_road['연장_mean'] = df_road['연장_mean'] / df_road['동별_총연장']
df_road = df_road.drop(columns=['동별_총연장', '시군구명'])
df_road

,동,도로등급,연장_mean,구
0,대명동,일반국도,0.326821,남구
1,대명동,특별광역시도,0.673179,남구
2,봉덕동,도시고속도로,0.299078,남구
3,봉덕동,연결로,0.264031,남구
4,봉덕동,특별광역시도,0.436891,남구
...,...,...,...,...
729,태평로3가,특별광역시도,1.000000,중구
730,포정동,특별광역시도,1.000000,중구
731,하서동,특별광역시도,1.000000,중구
732,향촌동,특별광역시도,1.000000,중구


In [716]:
result = {}
for idx, row in df_road.iterrows():
  if row['구'] in result.keys():
    if row['동'] in result[row['구']].keys():
      result[row['구']][row['동']][row['도로등급']] = row['연장_mean']
    else:
      result[row['구']][row['동']] = {
        row['도로등급']: row['연장_mean']
      }
  else:
    result[row['구']] = {
      row['동']: {
        row['도로등급']: row['연장_mean']
      }
    }

In [729]:
pd_result = []
for k,v in result.items():
  for k_2, v_2 in v.items():
    data = {'구': k}
    data['동'] = k_2
    for k_3, v_3 in v_2.items():
      data[k_3] = v_3 
    pd_result.append(data)

In [734]:
df_road = pd.DataFrame(pd_result).fillna(0)
df_road

,구,동,일반국도,특별광역시도,도시고속도로,연결로,고속도로,국가지원지방도,시군도로,지방도
0,남구,대명동,0.326821,0.673179,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,남구,봉덕동,0.000000,0.436891,0.299078,0.264031,0.000000,0.0,0.0,0.0
2,남구,이천동,1.409551,0.731295,0.446326,0.306516,1.120677,0.0,0.0,0.0
3,수성구,이천동,0.395049,0.204957,0.125090,0.085906,0.314088,0.0,0.0,0.0
4,수성구,가천동,0.395049,0.204957,0.000000,0.085906,0.314088,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
286,서구,원대동2가,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
287,서구,원대동3가,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
288,서구,이현동,0.000000,0.105524,0.177731,0.121068,0.595677,0.0,0.0,0.0
289,서구,중리동,0.000000,0.105524,0.177731,0.121068,0.595677,0.0,0.0,0.0


In [735]:
df_road.to_csv('../../main/data/origin/additionals/road_long_by_dong.csv')